In [1]:
import torch
import torchvision
from lib.data_handlers import Load_ImageNet100, Load_PACS
from overcomplete.models import DinoV2, ViT, ResNet
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import TopKSAE, train_sae
from overcomplete.visualization import (overlay_top_heatmaps, evidence_top_images, zoom_top_images, contour_top_image)
import os
import matplotlib.pyplot as plt
from einops import rearrange
from lib.universal_trainer import train_usae
from lib.activation_generator import Load_activation_dataloader
import torch.nn as nn
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
import timm
from torchsummary import summary
from lib.visualizer import visualize_concepts
from domainbed import algorithms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

checkpoint_ERM = torch.load(r"C:\Users\sproj_ha\Desktop\vision_interp\models\PACS\ERM_sketch\model.pkl")
checkpoint_IRM = torch.load(r"C:\Users\sproj_ha\Desktop\vision_interp\models\PACS\IRM_sketch\model.pkl")

model_ERM = algorithms.ERM(input_shape=checkpoint_ERM["model_input_shape"],hparams= checkpoint_ERM["model_hparams"],num_domains= checkpoint_ERM["model_num_domains"],num_classes= checkpoint_ERM["model_num_classes"])
model_IRM = algorithms.IRM(input_shape=checkpoint_IRM["model_input_shape"],hparams= checkpoint_IRM["model_hparams"],num_domains= checkpoint_IRM["model_num_domains"],num_classes= checkpoint_IRM["model_num_classes"])

model_ERM.load_state_dict(checkpoint_ERM["model_dict"])
model_IRM.load_state_dict(checkpoint_IRM["model_dict"])

c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [3]:
model_ERM.network

Sequential(
  (0): ResNet(
    (network): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act2): ReLU(inplace=True)
          (aa): Identity()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fals

In [16]:
x = torch.ones((1, 64, 224, 224)).to(device)
x = model_ERM.network[0].network.layer1(x)
x = model_ERM.network[0].network.layer2(x)
x = model_ERM.network[0].network.layer3(x)
x = model_ERM.network[0].network.layer4(x)

print(x.shape)

torch.Size([1, 2048, 28, 28])


In [3]:
models = {
    "ERM": model_ERM,
    "IRM": model_IRM
}

activation_dir = "activations/PACS_sketch_ResNet_ERM_IRM"
image_loader = Load_PACS(root_dir="C:/Users/sproj_ha/Desktop/vision_interp/datasets/data/PACS", batch_size=32)

activations_dataloader = Load_activation_dataloader(
    models=models,
    image_dataloader=image_loader,
    max_seq_len=None,   
    save_dir=activation_dir, 
    generate=False,  
    rearrange_string='n c w h -> (n w h) c'
    )

SAE Visualization

In [4]:
concepts = 2048 * 8
epochs = 50
lr=5e-5
sample = next(iter(activations_dataloader))

In [5]:
SAEs = {}
optimizers = {}
schedulers = {}

for key, model in models.items():
  SAEs[key] = TopKSAE(sample[f"activations_{key}"].shape[-1], nb_concepts=concepts, top_k=16, device='cuda')
  optimizers[key] = torch.optim.Adam(SAEs[key].parameters(), lr=lr)

  warmup_scheduler = LinearLR(optimizers[key], start_factor=1e-6 / 3e-4, end_factor=1.0, total_iters=10)
  cosine_scheduler = CosineAnnealingLR(optimizers[key], T_max=epochs, eta_min=1e-6)
  schedulers[key] = SequentialLR(optimizers[key], schedulers=[warmup_scheduler, cosine_scheduler], milestones=[10])


criterion = nn.L1Loss(reduction="mean")

In [ ]:
train_usae(names=list(models.keys()),
           models=SAEs,
           dataloader=activations_dataloader,
           criterion=criterion,
           nb_epochs=epochs,
           optimizers=optimizers,
           schedulers=schedulers,
           device='cuda')

Epoch 1/50: 100%|██████████| 313/313 [00:12<00:00, 24.30it/s, loss=0.483]



[Epoch 1] Loss: 153.3956 | Time: 12.88s | Dead Features: 0.0%


Epoch 2/50: 100%|██████████| 313/313 [00:12<00:00, 24.47it/s, loss=0.45] 



[Epoch 2] Loss: 139.5057 | Time: 12.79s | Dead Features: 0.0%


Epoch 3/50: 100%|██████████| 313/313 [00:12<00:00, 24.47it/s, loss=0.439]


...


[Epoch 48] Loss: 90.8270 | Time: 12.71s | Dead Features: 0.0%


Epoch 49/50: 100%|██████████| 313/313 [00:12<00:00, 24.76it/s, loss=0.278]



[Epoch 49] Loss: 90.6712 | Time: 12.64s | Dead Features: 0.0%


Epoch 50/50: 100%|██████████| 313/313 [00:12<00:00, 24.69it/s, loss=0.252]



[Epoch 50] Loss: 90.5260 | Time: 12.68s | Dead Features: 0.0%


In [8]:
model_state_dicts = {name: model.state_dict() for name, model in SAEs.items()}
torch.save(model_state_dicts, "./models/USAEs_ResNet_ERM_IRM.pt")

In [6]:
model_state_dicts = torch.load("./models/USAEs_ResNet_ERM_IRM.pt")
for name, model in SAEs.items():
    print(model.load_state_dict(model_state_dicts[name]))

<All keys matched successfully>
<All keys matched successfully>


In [ ]:
val_activation_dir = "activations/ImageNet100_Dino_ViT_VAL"
val_image_loader = Load_ImageNet100(root_dir=r"C:\Users\sproj_ha\Desktop\vision_interp\datasets\imagenet100\val.X", transform=None, batch_size=256, shuffle=True)

val_activations_dataloader = Load_activation_dataloader(
    models=models,
    image_dataloader=val_image_loader,
    max_seq_len=None,   
    save_dir=val_activation_dir, 
    generate=True,  
    rearrange_string='n c w h -> (n w h) c'
)

In [8]:
sample = next(iter(activations_dataloader))

sample['activations_IRM'].shape

torch.Size([1, 1568, 2048])

In [14]:
import matplotlib.pyplot as plt
import torch
from einops import rearrange
import heapq
from overcomplete.visualization.plot_utils import (
    get_image_dimensions,
    interpolate_cv2,
    show,
)
from overcomplete.visualization.cmaps import VIRIDIS_ALPHA
import itertools
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from copy import deepcopy
from torch.profiler import profile, record_function, ProfilerActivity



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def save_concept_plot(concept_id, save_dir):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

    filename = f"concept_{concept_id:04d}.png"
    filepath = save_dir / filename
    plt.savefig(filepath, bbox_inches="tight", dpi=300)


import time
import heapq
import itertools
from tqdm import tqdm
import torch
from einops import rearrange

def visualize_concepts2(
    activation_loader,
    SAEs,
    save_dir,
    patch_width,
    num_concepts,
    n_images=10,
    abort_threshold=0,
):
    import os
    os.makedirs(save_dir, exist_ok=True)

    print("Starting concept visualizer...\n")

    # Initialize a heap for each model that will store the (score, image)
    print("Initializing heaps...")
    start = time.time()
    heaps = {
        name: {concept_id: [] for concept_id in range(num_concepts)}
        for name in SAEs.keys()
    }
    print(f"Heap initialization time: {time.time() - start:.2f}s\n")

    for _, sae in SAEs.items():
        sae.eval()
        sae.to(device)

    counter = itertools.count()

    for batch_idx, batch in enumerate(tqdm(activation_loader)):
        print(f"\n--- Batch {batch_idx} ---")

        for name, sae in SAEs.items():
            print(f"Processing model: {name}")

            # Step 1: Encode
            start = time.time()
            with torch.no_grad():
                x = batch[f"activations_{name}"].squeeze().to(device)
                _, heatmaps = sae.encode(x)
            print(f"Encoding time: {time.time() - start:.2f}s")

            # Step 2: Rearranging heatmaps
            start = time.time()
            heatmaps = rearrange(
                heatmaps, "(n w h) d -> n w h d", w=patch_width, h=patch_width
            )  # shape: [B, w, h, D]
            print(f"Rearrange time: {time.time() - start:.2f}s")

            # Step 3: Move data to CPU
            start = time.time()
            images_cpu = batch["images"].squeeze().cpu()
            heatmaps_cpu = heatmaps.cpu()
            print(f"CPU transfer time: {time.time() - start:.2f}s")

            # Step 4: Compute mean heatmaps
            start = time.time()
            B, w, h, D = heatmaps.shape
            mean_heatmaps = heatmaps.mean(dim=(1, 2))  # shape: [B, D]
            print(f"Mean heatmap computation time: {time.time() - start:.2f}s")

            # Step 5: Concept-wise scoring and heap maintenance
            start = time.time()
            for concept_id in range(D):
                scores = mean_heatmaps[:, concept_id]  # [B]

                valid_mask = scores > abort_threshold
                if not valid_mask.any():
                    continue

                valid_scores = scores[valid_mask]
                valid_indices = valid_mask.nonzero(as_tuple=True)[0]

                top_scores, top_ranks = torch.topk(valid_scores, k=min(n_images, valid_scores.size(0)))
                top_indices = valid_indices[top_ranks]

                for score, idx in zip(top_scores, top_indices):
                    z = {
                        "image": images_cpu[idx],
                        "heatmap": heatmaps_cpu[idx, :, :, concept_id],
                    }
                    score_item = (-score.item(), next(counter), z)

                    heap = heaps[name][concept_id]
                    if len(heap) < n_images:
                        heapq.heappush(heap, score_item)
                    else:
                        heapq.heappushpop(heap, score_item)
            print(f"Heap update time: {time.time() - start:.2f}s")

    print("\nFinished processing all batches.")

    with ProcessPoolExecutor() as executor:
        args = ((concept, heaps, SAEs, n_images, save_dir) for concept in range(num_concepts))
        list(tqdm(executor.map(lambda p: worker(*p), args), total=num_concepts))


def worker(concept, heaps, SAEs, n_images, save_dir):
    if all(len(heaps[name][concept]) == 0 for name in SAEs.keys()):
        return
    else:
        subset = {key: deepcopy(heaps[key][concept]) for key in SAEs.keys()}
        plot_concept_grid(concept, subset, n_images, save_dir)


def plot_concept_grid(concept_id, subset, n_images, save_dir):
    """
    Plots and saves the visualization grid for a single concept.
    """
    rows_per_model = 2
    total_rows = len(subset) * rows_per_model
    fig = plt.figure(figsize=(n_images * 2.5, total_rows * 2.5))

    for j, (name, _) in enumerate(subset.items()):
        
        heap = subset[name]
        title_row = j * rows_per_model + 1
        image_row_start = title_row + 1

        # Title row
        title_ax = plt.subplot(total_rows, n_images, (title_row - 1) * n_images + 1)
        plt.axis("off")
        plt.text(
            0.5,
            0.5,
            name,
            ha="center",
            va="center",
            fontsize=14,
            weight="bold",
            transform=title_ax.transAxes,
        )

        # Image rows
        for i in range(n_images):
            
            if heap:
                idx = (image_row_start - 1) * n_images + i + 1                
                _, _, z = heapq.heappop(heap)
                image, heat = z["image"], z["heatmap"]

                width, height = get_image_dimensions(image)
                heatmap = interpolate_cv2(heat, (width, height))

                ax = plt.subplot(total_rows, n_images, idx)
                ax.axis("off")
                show(image)
                show(heatmap, cmap=VIRIDIS_ALPHA, alpha=1.0)
            
            else:
                ax = plt.subplot(total_rows, n_images, idx)
                ax.axis("off")
                show(torch.zeros(3, 224, 224))

    plt.tight_layout()
    save_concept_plot(concept_id, save_dir)
    plt.close()

In [15]:
visualize_concepts2(
        activation_loader=activations_dataloader,
        SAEs=SAEs,
        num_concepts=concepts,
        n_images=4,
        patch_width=7, 
        save_dir="results/visualizer_ResNet_ERM_IRM_V2/",
        abort_threshold=0.0,
    )

Starting concept visualizer...

Initializing heaps...
Heap initialization time: 0.00s



  0%|          | 0/313 [00:00<?, ?it/s]


--- Batch 0 ---
Processing model: ERM
Encoding time: 0.01s
Rearrange time: 0.00s
CPU transfer time: 0.16s
Mean heatmap computation time: 0.00s
Heap update time: 1.40s
Processing model: IRM
Encoding time: 0.00s
Rearrange time: 0.00s
CPU transfer time: 0.02s
Mean heatmap computation time: 0.00s


  0%|          | 1/313 [00:02<13:14,  2.54s/it]

Heap update time: 0.93s

--- Batch 1 ---
Processing model: ERM
Encoding time: 0.00s
Rearrange time: 0.00s
CPU transfer time: 0.06s
Mean heatmap computation time: 0.00s
Heap update time: 1.45s
Processing model: IRM
Encoding time: 0.00s
Rearrange time: 0.00s
CPU transfer time: 0.06s
Mean heatmap computation time: 0.00s


...

Heap update time: 0.88s

--- Batch 312 ---
Processing model: ERM
Encoding time: 0.00s
Rearrange time: 0.00s
CPU transfer time: 0.03s
Mean heatmap computation time: 0.00s
Heap update time: 1.42s
Processing model: IRM
Encoding time: 0.00s
Rearrange time: 0.00s
CPU transfer time: 0.08s
Mean heatmap computation time: 0.00s


100%|██████████| 313/313 [13:16<00:00,  2.54s/it]

Heap update time: 0.88s

Finished processing all batches.



  0%|          | 0/16384 [00:00<?, ?it/s]


AttributeError: Can't get local object 'visualize_concepts2.<locals>.<lambda>'

## Idea 1: Model Focused

#### Hypothesis: 
More generalizable models will have more domain-invariant concepts.  

#### Experiment: 
Take M different models (trained on different DG techniques), train a Universal Sparse Autoencoder to obtain a shared concept space. Identify a shared concept. Then identify if the models still activate that concept on similar OOD data (for e.g If the concept is the leg of the dog, then that concept should also be activated on a sketch of a deg). This will determine if the concept is domain-invariant for a particular model. We can then attribute this invariance to the DG method (e.g DAN, MMD etc used).

#### End Goal: 
A new metric/framework/insight for DG method evaluation.


## Idea 2: Concept Focused

#### Hypothesis:
Spurious Concepts will contribute more to classification decision for a less generalizable model.
Domain Invariant Concepts will contribute more to classification for a generalizable model.

#### Experiment:
Take M different models (trained on different DG techniques), train a Universal Sparse Autoencoder to obtain a shared concept space. Then identify which concepts influence a classification decision using some Mutual Information Proxy. Then identify which concepts are spurious (e.g capturing grass for a cow) by comparing label-concept correlations across multiple domains. 

#### End Goal:
Explore the relationship of spurious and invariant concepts with softmax distributions for different DG techniques. 


## Idea 3: Single Model
#### Hypothesis
There exists a tradeoff between Discrimination & Generalization

#### Experiment
Take one model, use a SAE to obtain a concept space. First identify which parts of the feature map led to a classification decision. Then identify what the concepts were activated by that part of the feature map. Finally, identify through perturbations, how the generalizable and discriminative the model is (through concept invariance, and classification accuracy respectively).

#### End Goal:
Plug and Play system where you input model, and it tells you how generalizable it is.
